# Data serialization

The qtt package contains methods to serialize objects to JSON.

In [9]:
import numpy as np
from functools import partial
from dataclasses_json import dataclass_json
from dataclasses import dataclass
from qtt.utilities.json_serializer import encode_json, decode_json, encode_json_dataclass, decode_json_dataclass, qtt_serializer

In [2]:
json = encode_json([1.0, 'hello'])
print(f'list of items: {json}')
json = encode_json(np.array([1.,2.]))
print(f'numpy array: {json}')

list of items: [1.0, "hello"]
numpy array: {"__object__": "array", "__content__": {"__ndarray__": "AAAAAAAA8D8AAAAAAAAAQA==", "__data_type__": "<f8", "__shape__": [2]}}


In [3]:
decoded_array = decode_json(json)
print(decoded_array)

[1. 2.]


## Custom data types

Custom data objects can be serialized by creating an encoder and decoder. For example to serialize `dataclass` objects with JSON we can do the following.

In [4]:
@dataclass_json
@dataclass
class CustomClass():
    
    x : float
    y : str

mydata = CustomClass(x=1., y='hi')
print(mydata)

CustomClass(x=1.0, y='hi')


Encoding a custom object is directly possible:

In [5]:
try:
    encode_json(mydata)
except TypeError as ex:
    print(f'could not encode object: {ex}')

could not encode object: Object of type 'CustomClass' is not JSON serializable


We can create custom encoding methods to make the serialization possible.

In [13]:
class_type=CustomClass

tag=f'_dataclass_{class_type.__name__}'
qtt_serializer.register(class_type, partial(encode_json_dataclass, tag), tag, partial(decode_json_dataclass, class_type))

json=encode_json(mydata)
print(f'encoded json: {json}')
decoded_object=decode_json(json)
print(f'decoded_object: {decoded_object}')

encoded json: {"__object__": "_dataclass_CustomClass", "__content__": {"x": 1.0, "y": "hi"}}
decoded_object: CustomClass(x=1.0, y='hi')
